In [60]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [61]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [70]:
# load IMDB dataset
max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# print the shape data for train and test sets
print(f'Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {x_test.shape}, Testing labels shape: {y_test.shape}')


Training data shape: (25000,), Training labels shape: (25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [63]:
len(x_train[4])

147

In [64]:
# inspect sample review and its label
sample_review = x_train[0]
sample_label = y_train[0]
print(f'Review: {sample_review}')
print(f'Label: {sample_label}')

Review: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Label: 1


In [65]:
# mapping of words to their indices
word_index = imdb.get_word_index()
word_index 
# joining the index and words to see sentence meaning f
reverse_word_index = {value: key for (key, value) in word_index.items()}
decoded_review = " ".join([reverse_word_index.get(i -3, '?') for i in sample_review])
print(f'Decoded review: {decoded_review}')

Decoded review: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have do

In [83]:
# sequence padding
maxlen = 500
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print(f'Padded training data shape: {x_train.shape}')
print(f'Padded testing data shape: {x_test.shape}')

Padded training data shape: (25000, 500)
Padded testing data shape: (25000, 500)


In [84]:
# train simple RNN model
dim = 128
maxlen = 500
model = Sequential()
model.add(Embedding(max_features,dim, input_length=maxlen)) # embedding layer
model.add(SimpleRNN(128, activation='relu'))  # simple RNN layer
model.add(Dense(1, activation='sigmoid'))

model.build((None, maxlen))   # force build with batch dim unspecified
model.summary()

c:\Users\USER\Documents\ML and NLP\simpleRNN\venvE\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_6 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [85]:
# create an instance of early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [86]:
# train the model with early stopping
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])   

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 171s 268ms/step - accuracy: 0.6461 - loss: 8.5865 - val_accuracy: 0.8154 - val_loss: 0.4184
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 195ms/step - accuracy: 0.7085 - loss: 3542419456.0000 - val_accuracy: 0.6468 - val_loss: 0.6000
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 116s 186ms/step - accuracy: 0.7514 - loss: 0.5317 - val_accuracy: 0.7234 - val_loss: 0.5284
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 188s 301ms/step - accuracy: 0.7630 - loss: 0.5147 - val_accuracy: 0.7164 - val_loss: 0.5353
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 169s 247ms/step - accuracy: 0.8386 - loss: 0.4017 - val_accuracy: 0.7816 - val_loss: 0.4706
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 171s 197ms/step - accuracy: 0.8827 - loss: 0.3300 - val_accuracy: 0.8018 - val_loss: 0.4376


In [87]:
model.save('simple_rnn_imdb_model.h5')

In [82]:
print("x_train.shape:", x_train.shape)      # expected (25000, 500)
print("x_train.ndim:", x_train.ndim)        # expected 2
print("x_train.dtype:", x_train.dtype)      # expected int32/int64, NOT object
print("y_train.shape:", y_train.shape)
print("y_train.dtype:", y_train.dtype)
print("sample element type:", type(x_train[0][0]), "sample values:", x_train[0][:10])

x_train.shape: (25000,)
x_train.ndim: 1
x_train.dtype: object
y_train.shape: (25000,)
y_train.dtype: int64
sample element type: <class 'int'> sample values: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
